<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/Finanse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install llama-index-llms-anthropic llama-index-tools-code-interpreter  llama-index -q

Instalacja trzech pakietów przy użyciu managera pip:

1. `llama-index-llms-anthropic` - moduł integrujący modele językowe Anthropic z frameworkiem LlamaIndex. LlamaIndex to narzędzie do tworzenia aplikacji wykorzystujących duże modele językowe.

2. `llama-index-tools-code-interpreter` - narzędzie do interpretacji i wykonywania kodu w ramach LlamaIndex. Pozwala na uruchamianie fragmentów kodu w różnych językach programowania.

3. `llama-index` - główny pakiet frameworka LlamaIndex, który służy do tworzenia aplikacji opartych o modele językowe, indeksowania danych i generowania odpowiedzi.

Flaga `-q` na końcu polecenia wymusza tryb "quiet", czyli ogranicza wyświetlanie komunikatów podczas instalacji do minimum - pokazywane są tylko błędy krytyczne.

Te trzy pakiety razem tworzą środowisko do budowania aplikacji wykorzystujących modele językowe Anthropic, z możliwością interpretacji kodu i zarządzania danymi.

In [ ]:
from google.colab import userdata
import os
from llama_index.tools.code_interpreter.base import CodeInterpreterToolSpec
from llama_index.llms.anthropic import Anthropic
from llama_index.core import Settings
from llama_index.core.agent import FunctionCallingAgent
import yfinance as yf
import pandas as pd
from datetime import datetime, date


Ten kod importuje niezbędne biblioteki i moduły:

1. `from google.colab import userdata` - import modułu do obsługi danych użytkownika w Google Colab.

2. `import os` - standardowa biblioteka Pythona do operacji na systemie operacyjnym.

3. `from llama_index.tools.code_interpreter.base import CodeInterpreterToolSpec` - import specyfikacji narzędzia do interpretacji kodu z LlamaIndex.

4. `from llama_index.llms.anthropic import Anthropic` - import modułu do komunikacji z modelami językowymi Anthropic.

5. `from llama_index.core import Settings` - import ustawień z głównego modułu LlamaIndex.

6. `from llama_index.core.agent import FunctionCallingAgent` - import agenta wykonującego funkcje z LlamaIndex.

7. `import yfinance as yf` - biblioteka do pobierania danych finansowych z Yahoo Finance.

8. `import pandas as pd` - biblioteka do analizy i manipulacji danymi.

9. `from datetime import datetime, date` - import klas do obsługi dat i czasu.

In [ ]:
class CFG:
  model = "claude-3-5-sonnet-20241022"

In [ ]:
os.environ["ANTHROPIC_API_KEY"] = userdata.get('claude')

Ta linia kodu ustawia klucz API dla usług Anthropic:

`os.environ["ANTHROPIC_API_KEY"]` ustawia zmienną środowiskową o nazwie "ANTHROPIC_API_KEY". Zmienne środowiskowe to zmienne systemowe dostępne dla wszystkich procesów w systemie.

`userdata.get('claude')` pobiera wartość klucza API zapisaną wcześniej w danych użytkownika Google Colab pod nazwą 'claude'.

Ten kod jest niezbędny do uwierzytelnienia się w API Anthropic - bez poprawnego klucza API nie będzie możliwe korzystanie z modeli językowych Claude. Jest to standardowy sposób przechowywania poufnych danych dostępowych, bezpieczniejszy niż wpisywanie klucza API bezpośrednio w kodzie.

# Agent

In [ ]:
code_spec = CodeInterpreterToolSpec()

tools = code_spec.to_tool_list()

Ta część kodu tworzy specyfikację narzędzia do interpretacji kodu i przygotowuje listę narzędzi:

`CodeInterpreterToolSpec()` tworzy nowy obiekt specyfikacji interpretera kodu. Jest to klasa, która definiuje jak kod powinien być interpretowany i wykonywany.

`code_spec.to_tool_list()` przekształca tę specyfikację w listę narzędzi, które będą używane przez agenta. Narzędzia te pozwalają na:
- Wykonywanie kodu Pythona
- Ocenę wyników
- Obsługę błędów
- Zarządzanie stanem wykonania

Otrzymana lista `tools` zawiera wszystkie funkcjonalności potrzebne do uruchamiania i analizy kodu w kontekście agenta LlamaIndex.

In [ ]:

tokenizer = Anthropic().tokenizer
Settings.tokenizer = tokenizer

llm_claude = Anthropic(model = CFG.model)


agent = FunctionCallingAgent.from_tools(
    tools,
    llm=llm_claude,
    verbose=True,
    allow_parallel_tool_calls=False,
)

Ta część kodu konfiguruje tokenizer i tworzy agenta do wykonywania funkcji:

`tokenizer = Anthropic().tokenizer` pobiera tokenizer z instancji Anthropic. Tokenizer to narzędzie, które zamienia tekst na tokeny (mniejsze jednostki) zrozumiałe dla modelu językowego.

`Settings.tokenizer = tokenizer` ustawia pobrany tokenizer jako domyślny w globalnych ustawieniach.

`llm_claude = Anthropic(model = CFG.model)` tworzy instancję modelu Claude, używając konfiguracji zdefiniowanej w CFG.model.

Ostatni fragment tworzy agenta funkcyjnego:
- `FunctionCallingAgent.from_tools(tools)` - tworzy agenta z wcześniej zdefiniowanej listy narzędzi
- `llm=llm_claude` - przypisuje mu model Claude
- `verbose=True` - włącza szczegółowe logowanie
- `allow_parallel_tool_calls=False` - wyłącza równoległe wykonywanie funkcji

Ten agent będzie odpowiedzialny za wykonywanie zadań i interakcję z modelem językowym.

In [ ]:
stock = 'TESLA'

prompt = f"""
Write a python code to :
- Detect which date is today
- Based on this date, fetch historical prices of {stock} from the beginning of the month until today.
- Analyze last month prices
"""

resp = agent.chat(prompt)

> Running step 6f264894-e8d8-4593-a2fc-207ca264a7c8. Step input: 
Write a python code to :
- Detect which date is today
- Based on this date, fetch historical prices of TESLA from the beginning of the month until today.
- Analyze the last month prices

Added user message to memory: 
Write a python code to :
- Detect which date is today
- Based on this date, fetch historical prices of TESLA from the beginning of the month until today.
- Analyze the last month prices

=== LLM Response ===
I'll help you write a Python code to analyze Tesla's stock prices. We'll use `yfinance` for fetching stock data and `pandas` for analysis. Here's the code:
=== Calling Function ===
Calling function: code_interpreter with args: {}
=== Function Output ===
Encountered error: CodeInterpreterToolSpec.code_interpreter() missing 1 required positional argument: 'code'
> Running step 12d5231a-0d45-4e92-a247-63627f30a56e. Step input: None
=== LLM Response ===
Let me write and execute the code:
=== Calling Functio

Ten fragment kodu tworzy zapytanie o analizę cen akcji Tesli i wysyła je do agenta:

`stock = 'TESLA'` definiuje zmienną przechowującą nazwę spółki do analizy.

`prompt` to tekst polecenia dla agenta, który prosi o:
- Wykrycie dzisiejszej daty
- Pobranie historycznych cen Tesli od początku miesiąca do dziś
- Analizę cen z ostatniego miesiąca

Format f-string `f"""..."""` pozwala na wstawienie zmiennej `stock` do tekstu.

`agent.chat(prompt)` wysyła to zapytanie do agenta, który:
- Przetwarza tekst polecenia
- Generuje odpowiedni kod w Pythonie
- Wykonuje go
- Zwraca wyniki

Rezultat zostaje zapisany w zmiennej `resp`, która będzie zawierać odpowiedź agenta wraz z wynikami analizy.

In [ ]:
print(resp.response)

I've created a comprehensive analysis of Tesla's stock performance for the current month. Let me break down the results:

1. Date Information:
   - Today's date is November 22, 2024
   - Analysis period: November 1, 2024 - November 22, 2024

2. Price Statistics:
   - Average price: $310.46
   - Lowest price: $242.84
   - Highest price: $350.00
   - Price volatility (std dev): $36.75

3. Price Movement:
   - Starting price: $248.98
   - Current price: $339.64
   - Total gain: $90.66
   - Percentage gain: 36.41%

4. Volume Analysis:
   - Average daily trading volume: 116.6 million shares
   - Highest volume day: November 11, 2024, with 210.5 million shares traded

5. Price Extremes:
   - Highest price reached: $358.64 on November 11, 2024
   - Lowest price reached: $238.88 on November 4, 2024

The analysis shows that Tesla stock has had a very strong performance this month with a significant increase of 36.41%. The stock showed considerable volatility, with a range of about $120 between 

Access to the Agent Memory

In [ ]:
agent.memory

ChatMemoryBuffer(chat_store=SimpleChatStore(store={'chat_history': [ChatMessage(role=<MessageRole.USER: 'user'>, content='\nWrite a python code to :\n- Detect which date is today\n- Based on this date, fetch historical prices of TESLA from the beginning of the month until today.\n- Analyze the last month prices\n', additional_kwargs={}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Here is the Python code to accomplish the requested tasks:', additional_kwargs={'tool_calls': [{'id': 'toolu_01GNksfRTm4r2rrzBTTntY7V', 'input': {'code': 'import datetime\nimport yfinance as yf\n\n# Detect today\'s date\ntoday = datetime.date.today()\nprint(f"Today\'s date is: {today}")\n\n# Fetch historical prices of TESLA from the beginning of the month until today\nstart_date = datetime.date(today.year, today.month, 1)\nend_date = today\ntesla = yf.Ticker("TSLA")\ntesla_data = tesla.history(start=start_date, end=end_date)\nprint(f"Historical prices of TESLA from {start_date} to {end_dat

Kontrola podstawą zaufania:

In [ ]:

# Get today's date
today = date.today()
print(f"Today's date is: {today}")

# Get the first day of the current month
start_date = today.replace(day=1)
print(f"Start date is: {start_date}")

# Fetch Tesla stock data
tesla = yf.download('TSLA', start=start_date, end=today)
print("\nTesla stock data shape:", tesla.shape)

# Basic analysis of the stock prices
print("\nPrice Summary Statistics:")
print(tesla['Close'].describe())

# Calculate daily returns
tesla['Daily_Return'] = tesla['Close'].pct_change()

# Print key metrics
print("\nKey Metrics:")
print(f"Starting Price: ${float(tesla['Close'].iloc[0]):.2f}")
print(f"Ending Price: ${float(tesla['Close'].iloc[-1]):.2f}")
print(f"Highest Price: ${float(tesla['High'].max()):.2f}")
print(f"Lowest Price: ${float(tesla['Low'].min()):.2f}")
print(f"Average Daily Return: {float(tesla['Daily_Return'].mean()*100):.2f}%")
print(f"Daily Return Volatility: {float(tesla['Daily_Return'].std()*100):.2f}%")

# Calculate total return for the period
total_return = float(((tesla['Close'].iloc[-1] - tesla['Close'].iloc[0]) / tesla['Close'].iloc[0]) * 100)
print(f"\nTotal Return for the period: {total_return:.2f}%")

[*********************100%***********************]  1 of 1 completed

Today's date is: 2024-11-22
Start date is: 2024-11-01

Tesla stock data shape: (15, 6)

Price Summary Statistics:
Ticker        TSLA
count    15.000000
mean    310.463998
std      36.754431
min     242.839996
25%     292.720001
50%     321.220001
75%     339.190002
max     350.000000

Key Metrics:
Starting Price: $248.98
Ending Price: $339.64
Highest Price: $358.64
Lowest Price: $238.88
Average Daily Return: 2.39%
Daily Return Volatility: 5.76%

Total Return for the period: 36.41%



<ipython-input-19-a69ed7d74f48>:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Starting Price: ${float(tesla['Close'].iloc[0]):.2f}")
<ipython-input-19-a69ed7d74f48>:23: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Ending Price: ${float(tesla['Close'].iloc[-1]):.2f}")
<ipython-input-19-a69ed7d74f48>:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Highest Price: ${float(tesla['High'].max()):.2f}")
<ipython-input-19-a69ed7d74f48>:25: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Lowest Price: ${float(tesla['Low'].min()):.2f}")
<ipython-input-19-a69ed7d74f48>:30